In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

In [7]:
# Load the dataset
train = pd.read_csv('./arcene_train.data',header=None,sep=' ')
del train[10000] ## Some problem in reading
train=train.as_matrix() ## Convert to numpy array

In [8]:
## Read Lables
lines = open('./arcene_train.labels').read().splitlines()
train_lables=lines

In [9]:
########## HELPER FUNCTIONS
def rbfkernel(gamma, distance):
    return np.exp(-gamma * distance)

def inverse_squareform(matrix):
    inv_sqfrm = []
    for i in range(len(matrix)):
        for j in range(i+1, len(matrix[i])):
            inv_sqfrm.append(matrix[i][j])
    inv_sqfrm = np.array(inv_sqfrm)
    return inv_sqfrm

def find_distance_matrix(data):
    euclid_distance = []
    for i in data:
        distance = []
        for j in data:
            distance.append(np.linalg.norm(i - j) * np.linalg.norm(i - j))
        distance = np.array(distance)
        euclid_distance.append(distance)
    euclid_distance = np.array(euclid_distance)
    return euclid_distance

In [10]:
# ## Build POLYNOMIAL KERNEL Function
# gamma=0.0001
# K=np.zeros(shape=(100,100))
# for i in range(0,len(train)):
#     for j in range(0,len(train)):
#         k = 1+ gamma*np.dot(train[i].T,train[j]) ## K(i,j) = ( 1 + x(i).T . x(j) )^p
#         k=math.pow(k,2)
#         K[i][j]=k

## Build Kernel Matrix
# calculate euclidean distance matrix
distance_matrix = find_distance_matrix(train)

# find variance of one dimensional distance list
variance = np.var(inverse_squareform(distance_matrix))

# calculate kernel (using rbfkernel)
gamma = 1/(2*variance)
K = rbfkernel(gamma, distance_matrix)

In [11]:
## Centre The Matrix
mean=K.mean(0)
K_centred=K-mean

In [12]:
## Find eign values, divide vecotrs by root lamba, sort ,pick k highest
tmp=np.linalg.eig(K_centred) ## Using Centred Matrix to find eign values
eign_values=tmp[0]
eign_vector=tmp[1]
rootlmda=np.sqrt(eign_values)
eign_vector=eign_vector/rootlmda

In [13]:
## Choose 'k' biggest eign vectors
num_of_comp=100
indexes = eign_values.argsort()[::-1]
alpha = eign_vector[:, indexes[0:num_of_comp]]

In [14]:
## Find the transformed data
Y=np.dot(K,alpha)

In [15]:
import matplotlib.pyplot as plt
plt.scatter(Y[:,0], Y[:,1])
plt.show()

In [11]:
# ##  Tally Results with Scikit
# from sklearn.decomposition import PCA, KernelPCA
# # Reduce
# kpca = KernelPCA(kernel="poly", degree=2,n_components=2)
# X_kpca = kpca.fit(train)
# Z = kpca.transform(train)
# # Plot
# plt.scatter(Z[:,0], Z[:,1])
# plt.show()

In [37]:
########################################## PCA IS COMPLETE ###################################

In [21]:
######### NOW DO FOR TEST DATA ################
## Load the dataset
test = pd.read_csv('./arcene_valid.data',header=None,sep=' ')
del test[10000] ## Some read problem wasthere
test=test.as_matrix() ## Convert to numpy array

## Read Lables
lines = open('./arcene_valid.labels').read().splitlines()
test_lables=lines

In [22]:
### Polynomial Kernel Matrix
# Ktest=np.zeros(shape=(100,100))
# for i in range(0,len(test)):
#     for j in range(0,len(train)):
#         ktest = 1+gamma*np.dot(test[i].T,train[j]) ## K(i,j) = ( 1 + x(i).T . x(j) )^p
#         ktest=math.pow(ktest,2)
#         Ktest[i][j]=ktest

### By RBF Kernel
Ktest= []
for i in test:
        dist = np.array([np.sum((i - row)**2) for row in train])
        k = rbfkernel(gamma, dist)
        kernel = np.dot(k, alpha)
        Ktest.append(kernel)


## Find the transformed data
# Ytest=np.dot(Ktest,alpha)
Ytest=Ktest

In [23]:
import matplotlib.pyplot as plt
plt.scatter(Ytest[:,0], Ytest[:,1])
plt.show()

TypeError: list indices must be integers or slices, not tuple

In [17]:
############ Now Applying SVM ###############

In [24]:
############ APPLYING SVM ####################
from sklearn import svm
# clf = svm.SVC(kernel='rbf',gamma=0.01) ## use linear too
clf = svm.SVC(kernel='linear') ## use linear too
clf.fit( Y,train_lables) ## Y is the transformed data
clf.score(Ytest,test_lables) ## Ytest is transformed test data

0.56000000000000005

In [25]:
# from sklearn.decomposition import PCA, KernelPCA
# kpca = KernelPCA(kernel="poly",degree=2, n_components=10,gamma=0.0001)
# Train_KPCA1 = kpca.fit_transform(train)
# Test1 = kpca.transform(train)
# Test1[0]